# Adaptive Sampling

In this tutorial you will learn:
    
    ...
    
    

## Some of the software we use:

* **Bluesky RunEngine** for experiment orchestration (sequencing)
* **Bluesky Ophyd** for device integration (for this demo, a simulated detector)
* **Bluesky Widgets** components for live-updating ("streaming") visualization
* **Matplotlib** for visualization
* **Bluesky Adaptive**, an adaptive "harness" for integrating an Agent in a feedback loop with the Bluesky RunEngine
* **Tensorflow** for the model

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt

In [ ]:
from bluesky import RunEngine
from bluesky.plans import count
from utils.simulated_hardware import detector, sample_selector, select_sample
from utils.visualization import stream_to_figures
from utils.adaptive_recommendations import with_agent

## Make a "RunEngine"

* Processes a set of instructions (a "plan") from the user
* Direct hardware, tracks what is moving when, and tries to clean up correct in the event of success or failures
* Emits metadata and data in a streaming fashion for consumers (plots, models, storage, etc.)

In [ ]:
RE = RunEngine()

The RunEngine can move this things, and it can take data.

## Acquire some images

In [ ]:
RE(select_sample(0))  # This moves a sample positioner to place Sample 0 in the beam.

Set up a Figure which we will stream data into.

In [ ]:
fig, axes = plt.subplots(squeeze=False, constrained_layout=True, figsize=(5, 5))
callback = stream_to_figures(fig, axes)

Take data. The figure above will show the image.

In [ ]:
RE(count([sample_selector, detector]), callback)

This sample happens to be "good". It gives a strong signal (high signal-to-noise ratio). A single exposure produces an image with sufficient contrast to interpret scientifically.

Now, move to the next sample, set up a new figure, and acquire another image.

In [ ]:
RE(select_sample(1))

In [ ]:
fig, axes = plt.subplots(squeeze=False, constrained_layout=True, figsize=(5, 5))
callback = stream_to_figures(fig, axes, start_at=1)

In [ ]:
RE(count([sample_selector, detector]), callback)

This sample happens to be "bad". It gives a weak signal (low signal-to-noise ratio). A single exposure does not produce sufficient contrast, but we can take additional exposures. The visualization above will display the average of all the exposures of this sample.

In [ ]:
RE(count([sample_selector, detector]), callback)

## Acquire images for more than one sample

Here, we make a Figure that display images for both the first (good) and second (bad) sample. As we acquire images, we will see them side by side here.

In [ ]:
fig, axes = plt.subplots(1, 2, squeeze=False, constrained_layout=True, figsize=(5, 3))
callback = stream_to_figures(fig, axes)

In [ ]:
RE(select_sample(0))

In [ ]:
RE(count([sample_selector, detector]), callback)

In [ ]:
RE(select_sample(1))

In [ ]:
RE(count([sample_selector, detector]), callback)

In [ ]:
RE(count([sample_selector, detector]), callback)

## Write a custom Bluesky "plan" to sweep samples

In [ ]:
def sequential_sweep(total_shots):
    "Sweep over the samples in order. Take up to `total_shots` shots."
    for shot in range(total_shots):
        yield from select_sample(shot % 9)
        yield from count([sample_selector, detector])

In [ ]:
fig, axes = plt.subplots(3, 3, constrained_layout=True, figsize=(5, 5))
callback = stream_to_figures(fig, axes)

Our simulated detector has a simulated "delay", standing in for the exposure and readout time in a real detector. Here we'll make it go faster so the following examples runs faster.

In [ ]:
detector.delay = 0.1

In [ ]:
unique_ids = RE(sequential_sweep(total_shots=18), callback)

## Use `bluesky-adaptive` to let an Agent drive the experiment

It is have access to the data as it is acquired and use this to decide when to move to the next sample.

These particular agents are aware of a "budget" of time for this experiment. They aim to make the most efficient use of the available time to obtain high-constrant, interpretable images.

### Agent 1: Naive Agent

This will do effectively same thing we just did above---sequential sweeps---but it will do so using the `bluesky-adaptive` machinery.

In [ ]:
fig, axes = plt.subplots(3, 3, constrained_layout=True, figsize=(5, 5))
callback = stream_to_figures(fig, axes)

In [ ]:
from utils.adaptive_recommendations import NaiveAgent

unique_ids = RE(
    with_agent(NaiveAgent(9), max_shots=50),
    callback,
)

### Agent 2: Reinforcement Learning Agent



In [ ]:
fig, axes = plt.subplots(3, 3, constrained_layout=True, figsize=(5, 5))
callback = stream_to_figures(fig, axes)

In [ ]:
from utils.adaptive_recommendations import RLAgent

unique_ids = RE(
    with_agent(RLAgent(9, 'tf_models/bluesky-tutorial/saved_models'), max_shots=50),
    callback,
)

### Agent 3: "Cheating" (Omniscient) Agent

This agent is told *a priori* which samples are good and which are bad.

In [ ]:
fig, axes = plt.subplots(3, 3, constrained_layout=True, figsize=(5, 5))
callback = stream_to_figures(fig, axes)

In [ ]:
from utils.adaptive_recommendations import CheatingAgent

unique_ids = RE(
    with_agent(CheatingAgent(9), max_shots=50),
    callback,
)